In [20]:
import networkx as nx
import freeman as fm
import matplotlib.pyplot as plt
from pprint import pprint
import pymysql
import sys

In [21]:
from setup import mysql_setup
conn = mysql_setup()

In [22]:
g = nx.Graph()


In [23]:
with conn.cursor() as cursor:
    try:
        cursor.execute("SELECT * FROM Playlist")
        playlists = cursor.fetchall()
    except pymysql.err.IntegrityError as e:
        print("Erro: não foi possivel dar select em playlist")
        pass

In [24]:
def return_tracks(id_playlist):
    with conn.cursor() as cursor:
        try:
            cursor.execute("SELECT * FROM Playlist_Track WHERE id_playlist = %s", (id_playlist))
            tracks = cursor.fetchall()
        except pymysql.err.IntegrityError as e:
            print("Erro: não foi possivel dar select em playlist")
            sys.exit()
    return tracks

In [25]:
for playlist in playlists:
    tracks = return_tracks(playlist[0])
    g.add_node(playlist[1])

for playlists_index1 in range(len(playlists)-1):
    tracks_1 = return_tracks(playlists[playlists_index1][0])
    
    if len(tracks_1) == 0:
        continue
        
    for playlists_index2 in range(len(playlists)-1-playlists_index1):
        tracks_2 = return_tracks(playlists[playlists_index1+playlists_index2+1][0])
        
        if len(tracks_2) == 0:
            continue

        count = 0
        
        for t1 in tracks_1:
            for t2 in tracks_2:
                if t1[1] == t2[1]:
                    count += 1
    
        g.add_edge(playlists[playlists_index1][1], playlists[playlists_index1+playlists_index2+1][1] , weight=count)

0
0
0
0
0
0
100
0
64
0
0
0
0
0
0
0
95
0
0
0
0
0
0
0
0
0
100
50
0
0
0
0
0
0
0
0
0
0
0
0
60
0
0
0
0
0
39
0
0
0
0
45
50
26
70
0
0
0
0
0
100
0
82
0
27
0
0
0
62
0


In [12]:
nx.write_gml(g, "test.gml")

In [13]:
g = fm.load('test.gml')

C:\Users\LiuSeeker\Desktop\6o-semestre\redes-sociais\redes-projeto1\spotify\freeman\__init__.py:68: UserWarning:

node pos is not allowed in file, ignoring



In [14]:
g.set_all_nodes('labpos', 'hover')
g.move('kamada_kawai')
g.interact()